# Colocating Sentinel-3 OLCI/SRAL and Sentinal-2 Optical Data
In this section, we embark on a detailed exploration of colocating Sentinel-3 data with Sentinel-2 optical data. Colocation of data from these two satellite missions enables a powerful synergy, harnessing the high spatial resolution of Sentinel-2 and the comprehensive coverage and colocated altimeter data from Sentinel-3. This fusion of datasets provides a richer, more detailed perspective of Earth's surface.

In the following sections, we will guide you through the necessary steps to identify and align these datasets.

Week 4 Materials are available [here](https://drive.google.com/drive/folders/1CZHnDj2DJic-e8ZYHOnjLpQejeX7-cOS?usp=drive_link).

## Step 0: Read in Functions Needed

To streamline our data fetching and processing, we'll first load the essential functions. These functions are identical to what we have for the data_fetching notebook in week 3. These functions essentially help you get metadata for the 2 satellites you care about.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime, timedelta
from shapely.geometry import Polygon, Point, shape
import numpy as np
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import os
import json
import folium


def make_api_request(url, method="GET", data=None, headers=None):
    global access_token
    if not headers:
        headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request(method, url, json=data, headers=headers)
    if response.status_code in [401, 403]:
        global refresh_token
        access_token = refresh_access_token(refresh_token)
        headers["Authorization"] = f"Bearer {access_token}"
        response = requests.request(method, url, json=data, headers=headers)
    return response


def query_sentinel3_olci_arctic_data(start_date, end_date, token):
    """
    Queries Sentinel-3 OLCI data within a specified time range from the Copernicus Data Space,
    targeting data collected over the Arctic region.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.

    Returns:
    DataFrame: Contains details about the Sentinel-3 OLCI images.
    """

    all_data = []
    arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"
    # arctic_polygon = (
    #     "POLYGON ((-81.7 71.7, -81.7 73.8, -75.1 73.8, -75.1 71.7, -81.7 71.7))"
    # )

    filter_string = (
        f"Collection/Name eq 'SENTINEL-3' and "
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'OL_1_EFR___') and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(next_url, headers=headers)
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def get_access_and_refresh_token(username, password):
    """Retrieve both access and refresh tokens."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "cdse-public",
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    tokens = response.json()
    return tokens["access_token"], tokens["refresh_token"]


def refresh_access_token(refresh_token):
    """Attempt to refresh the access token using the refresh token."""
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": "cdse-public",
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # This will throw an error for non-2xx responses
        return response.json()["access_token"]
    except requests.exceptions.HTTPError as e:
        print(f"Failed to refresh token: {e.response.status_code} - {e.response.text}")
        if e.response.status_code == 400:
            print("Refresh token invalid, attempting re-authentication...")
            # Attempt to re-authenticate
            username = username
            password = password
            # This requires securely managing the credentials, which might not be feasible in all contexts
            access_token, new_refresh_token = get_access_and_refresh_token(
                username, password
            )  # This is a placeholder
            refresh_token = (
                new_refresh_token  # Update the global refresh token with the new one
            )
            return access_token
        else:
            raise

def download_single_product(
    product_id, file_name, access_token, download_dir="downloaded_products"
):
    """
    Download a single product from the Copernicus Data Space.

    :param product_id: The unique identifier for the product.
    :param file_name: The name of the file to be downloaded.
    :param access_token: The access token for authorization.
    :param download_dir: The directory where the product will be saved.
    """
    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Construct the download URL
    url = (
        f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    )

    # Set up the session and headers
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)

    # Perform the request
    response = session.get(url, headers=headers, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Define the path for the output file
        output_file_path = os.path.join(download_dir, file_name + ".zip")

        # Stream the content to a file
        with open(output_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {output_file_path}")
    else:
        print(
            f"Failed to download product {product_id}. Status Code: {response.status_code}"
        )

def query_sentinel3_sral_arctic_data(start_date, end_date, token):
    """
    Queries Sentinel-3 SRAL data within a specified time range from the Copernicus Data Space,
    targeting data collected over the Arctic region.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.

    Returns:
    DataFrame: Contains details about the Sentinel-3 SRAL images.
    """

    all_data = []
    # arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"
    arctic_polygon = (
        "POLYGON ((-81.7 71.7, -81.7 73.8, -75.1 73.8, -75.1 71.7, -81.7 71.7))"
    )

    filter_string = (
        f"Collection/Name eq 'SENTINEL-3' and "
        f"Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'SR_2_LAN_SI') and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(
            next_url, headers={"Authorization": f"Bearer {token}"}
        )
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def query_sentinel2_arctic_data(
    start_date,
    end_date,
    token,
    min_cloud_percentage=10,
    max_cloud_percentage=50,
):
    """
    Queries Sentinel-2 data within a specified time range from the Copernicus Data Space,
    considering a range of cloud coverage by treating greater than and less than conditions as separate attributes.
    Handles pagination to fetch all available data.

    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    token (str): Access token for authentication.
    min_cloud_percentage (int): Minimum allowed cloud coverage.
    max_cloud_percentage (int): Maximum allowed cloud coverage.

    Returns:
    DataFrame: Contains details about the Sentinel-2 images.
    """

    all_data = []
    arctic_polygon = "POLYGON((-180 60, 180 60, 180 90, -180 90, -180 60))"

    filter_string = (
        f"Collection/Name eq 'SENTINEL-2' and "
        f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/Value ge {min_cloud_percentage}) and "
        f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/Value le {max_cloud_percentage}) and "
        f"ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T23:59:59.999Z"
    )

    next_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={filter_string} and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{arctic_polygon}')&"
        f"$top=1000"
    )

    headers = {"Authorization": f"Bearer {token}"}

    while next_url:
        response = make_api_request(
            next_url, headers={"Authorization": f"Bearer {token}"}
        )
        if response.status_code == 200:
            data = response.json()["value"]
            all_data.extend(data)
            next_url = response.json().get("@odata.nextLink")
        else:
            print(f"Error fetching data: {response.status_code} - {response.text}")
            break

    return pd.DataFrame(all_data)


def plot_results(results):
    m = folium.Map(location=[0, 0], zoom_start=2)
    for idx, row in results.iterrows():
        try:
            geojson1 = json.loads(row["Satellite1_Footprint"].replace("'", '"'))
            geojson2 = json.loads(row["Satellite2_Footprint"].replace("'", '"'))

            folium.GeoJson(geojson1, name=row["Satellite1_Name"]).add_to(m)
            folium.GeoJson(geojson2, name=row["Satellite2_Name"]).add_to(m)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

    folium.LayerControl().add_to(m)
    return m


def parse_geofootprint(footprint):
    """
    Parses a JSON-like string to extract the GeoJSON and convert to a Shapely geometry.
    """
    try:
        geo_json = json.loads(footprint.replace("'", '"'))
        return shape(geo_json)
    except json.JSONDecodeError:
        return None


def check_collocation(
    df1, df2, start_date, end_date, time_window=pd.to_timedelta("1 day")
):

    collocated = []
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    for idx1, row1 in df1.iterrows():
        footprint1 = parse_geofootprint(row1["GeoFootprint"])
        if footprint1 is None:
            continue

        s1_start = row1["ContentDate.Start"]
        s1_end = row1["ContentDate.End"]

        if s1_end < start_date or s1_start > end_date:
            continue

        s1_start_adjusted = s1_start - time_window
        s1_end_adjusted = s1_end + time_window

        for idx2, row2 in df2.iterrows():
            footprint2 = parse_geofootprint(row2["GeoFootprint"])
            if footprint2 is None:
                continue

            s2_start = row2["ContentDate.Start"]
            s2_end = row2["ContentDate.End"]

            if s2_end < start_date or s2_start > end_date:
                continue
            if max(s1_start_adjusted, s2_start) <= min(s1_end_adjusted, s2_end):
                if footprint1.intersects(footprint2):
                    collocated.append(
                        {
                            "Satellite1_Name": row1["Name"],
                            "Satellite1_ID": row1["Id"],
                            "Satellite1_Footprint": row1["GeoFootprint"],
                            "Satellite2_Name": row2["Name"],
                            "Satellite2_ID": row2["Id"],
                            "Satellite2_Footprint": row2["GeoFootprint"],
                            "Overlap_Start": max(
                                s1_start_adjusted, s2_start
                            ).isoformat(),
                            "Overlap_End": min(s1_end_adjusted, s2_end).isoformat(),
                        }
                    )

    return pd.DataFrame(collocated)


def make_timezone_naive(dt):
    """Convert a timezone-aware datetime object to timezone-naive in local time."""
    return dt.replace(tzinfo=None)


## Step 1: Get the Metadata for satellites (Sentinel-2 and Sentinel-3 OLCI in this case)
In this example, we illustrate how we co-locate Sentinel-2 and Sentinel-3 OLCI by fetching the metadata first (the same way we did in week 3). Since we are trying to find co-location between 2 satellites, we fetch 2 tables of metadata, representing 2 satellites we care about. In this case, they are named as object
'sentinel3_olci_data' and 'sentinel2_data'.







In [ ]:
username = "vibhagaddi@gmail.com"
password = "PristineFamily@09"
access_token, refresh_token = get_access_and_refresh_token(username, password)
start_date = "2018-06-01"
end_date = "2018-06-02"
path_to_save_data = "/content/drive/MyDrive/GEOL0069/Week 4/" # Here you can edit where you want to save your metadata
s3_olci_metadata = query_sentinel3_olci_arctic_data(
    start_date, end_date, access_token
)

s2_metadata = query_sentinel2_arctic_data(
    start_date,
    end_date,
    access_token,
    min_cloud_percentage=0,
    max_cloud_percentage=10,
)

# You can also save the metadata
s3_olci_metadata.to_csv(
    path_to_save_data+"sentinel3_olci_metadata.csv",
    index=False,
)

s2_metadata.to_csv(
    path_to_save_data+"sentinel2_metadata.csv",
    index=False,
)

You can try to print them to see what these 2 metadata look like.

In [ ]:
from IPython.display import display

display(s3_olci_metadata)


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,82cd6fbb-46a9-3464-8ff4-a712858c77c6,S3A_OL_1_EFR____20180601T015234_20180601T01553...,application/octet-stream,912255669,2024-08-31T03:05:16.157000Z,2025-05-08T21:57:51.791792Z,2025-05-08T21:57:51.791792Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/01...,"[{'Value': 'f388055cc26392059a140ab43b06cfbc',...","{'Start': '2018-06-01T01:52:33.918947Z', 'End'...","geography'SRID=4326;POLYGON ((117.924 52.4613,...","{'type': 'Polygon', 'coordinates': [[[117.924,..."
1,application/octet-stream,d24129b0-b489-34d9-a9bc-2dafb6261680,S3A_OL_1_EFR____20180601T001135_20180601T00143...,application/octet-stream,890014775,2024-08-31T03:02:58.143000Z,2025-05-08T21:55:51.868871Z,2025-05-08T21:55:51.868871Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/01...,"[{'Value': 'e364d22200a66a70e6b2fca9d693d4e3',...","{'Start': '2018-06-01T00:11:34.751541Z', 'End'...","geography'SRID=4326;POLYGON ((143.172 52.4588,...","{'type': 'Polygon', 'coordinates': [[[143.172,..."
2,application/octet-stream,d1966c1a-bedc-3f87-b7b0-b5a5745a881c,S3A_OL_1_EFR____20180601T014334_20180601T01463...,application/octet-stream,818174086,2024-08-31T03:04:36.331000Z,2025-05-08T21:57:21.969896Z,2025-05-08T21:57:21.969896Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/01...,"[{'Value': '2a8c0401905d1cd822c4a9f47620468a',...","{'Start': '2018-06-01T01:43:33.918947Z', 'End'...","geography'SRID=4326;POLYGON ((121.409 83.7886,...","{'type': 'Polygon', 'coordinates': [[[121.409,..."
3,application/octet-stream,efd0f1d6-0465-3670-b56c-af4f667b257a,S3A_OL_1_EFR____20180601T014034_20180601T01433...,application/octet-stream,805152199,2024-08-31T03:04:23.886000Z,2025-05-08T21:57:19.001043Z,2025-05-08T21:57:19.001043Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/01...,"[{'Value': '658580bacd8775d7549cf22ccc24b6db',...","{'Start': '2018-06-01T01:40:33.918947Z', 'End'...",geography'SRID=4326;POLYGON ((-53.3268 85.7884...,"{'type': 'Polygon', 'coordinates': [[[-53.3268..."
4,application/octet-stream,4c6ab3f8-ea5e-3080-b867-927dbd631246,S3A_OL_1_EFR____20180601T000535_20180601T00083...,application/octet-stream,817410712,2024-08-31T03:01:52.507000Z,2025-05-08T21:55:53.280295Z,2025-05-08T21:55:53.280295Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/01...,"[{'Value': 'a64bce8891ef1a03147680eda4fb966a',...","{'Start': '2018-06-01T00:05:34.751541Z', 'End'...",geography'SRID=4326;MULTIPOLYGON (((180 69.749...,"{'type': 'MultiPolygon', 'coordinates': [[[[18..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,application/octet-stream,87b3f609-b730-30bb-9765-7d91b4cad913,S3B_OL_1_EFR____20180602T230525_20180602T23061...,application/octet-stream,245412499,2024-09-15T10:07:27.512000Z,2025-05-15T19:33:53.892944Z,2025-05-15T19:33:53.892944Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/02...,"[{'Value': '35fc08edd5dfc912edb08613c894c965',...","{'Start': '2018-06-02T23:05:24.804797Z', 'End'...","geography'SRID=4326;POLYGON ((-14.9691 75.374,...","{'type': 'Polygon', 'coordinates': [[[-14.9691..."
340,application/octet-stream,8bcdc90e-02cb-300e-bb13-8c1947f96926,S3B_OL_1_EFR____20180602T230619_20180602T23091...,application/octet-stream,808833424,2024-09-15T10:08:06.111000Z,2025-05-15T19:36:02.894903Z,2025-05-15T19:36:02.894903Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR___/2018/06/02...,"[{'Value': '587f6236b6df5e4594642e593be91cbe',...","{'Start': '2018-06-02T23:06:18.951714Z', 'End'...","geography'SRID=4326;POLYGON ((-15.0526 85.794,...","{'type': 'Polygon', 'coordinates': [[[-15.0526..."
341,application/octet-stream,c33682f2-f1d5-35e8-89a6-890650b0a100,S3B_OL_1_EFR____20180602T231

In [ ]:
from IPython.display import display

display(s2_metadata)


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,159afe10-6623-48f1-9e9c-8f7e01e54348,S2A_MSIL2A_20180601T102021_N0500_R065_T33VVG_2...,application/octet-stream,916047145,2024-03-13T04:50:54.107000Z,2024-03-13T05:04:25.509921Z,2025-07-04T00:22:40.631124Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A_N0500/2018/06/01/S2...,"[{'Value': '6172ec7b4eb518875ab622f0115987d6',...","{'Start': '2018-06-01T10:20:21.024000Z', 'End'...",geography'SRID=4326;POLYGON ((13.4203154089248...,"{'type': 'Polygon', 'coordinates': [[[13.42031..."
1,application/octet-stream,fde438d0-78c0-43e9-9344-5f47cbf9f1b0,S2B_MSIL1C_20180601T093029_N0500_R136_T35VPG_2...,application/octet-stream,792958593,2024-02-12T20:17:21.469000Z,2024-02-12T22:06:37.706813Z,2025-06-21T05:22:41.644628Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': '5a175b2e9eae9b22108d9b609261ef34',...","{'Start': '2018-06-01T09:30:29.024000Z', 'End'...",geography'SRID=4326;POLYGON ((28.8162812265457...,"{'type': 'Polygon', 'coordinates': [[[28.81628..."
2,application/octet-stream,ea293a96-1e9d-489c-a59f-b4b330d78f97,S2A_MSIL1C_20180601T001611_N0500_R059_T59WPT_2...,application/octet-stream,725286361,2024-03-09T08:49:18.779000Z,2024-03-09T08:58:46.267783Z,2025-06-18T14:34:18.975914Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': '9d15364b29ae173477bde24477bf4f29',...","{'Start': '2018-06-01T00:16:11.024000Z', 'End'...",geography'SRID=4326;POLYGON ((173.654264122047...,"{'type': 'Polygon', 'coordinates': [[[173.6542..."
3,application/octet-stream,9e10bfde-4675-4fe2-8b0f-3b3e8988180a,S2A_MSIL1C_20180601T165851_N0500_R069_T23XNJ_2...,application/octet-stream,596632961,2024-03-06T15:31:46.633000Z,2024-03-06T15:43:20.153137Z,2025-06-21T05:17:17.314674Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': '8e7358a802865e74d5c9eef147ca2f09',...","{'Start': '2018-06-01T16:58:51.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-45.001038 80.16...,"{'type': 'Polygon', 'coordinates': [[[-45.0010..."
4,application/octet-stream,2840b56d-d2e2-4cb1-8515-d145598bd6b4,S2B_MSIL1C_20180601T124659_N0500_R138_T31XFK_2...,application/octet-stream,127717787,2024-03-11T01:58:38.916000Z,2024-03-11T02:09:28.407175Z,2025-06-21T05:20:22.291886Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': 'fa5744b6c8037fa1bfea96b11952ec52',...","{'Start': '2018-06-01T12:46:59.024000Z', 'End'...",geography'SRID=4326;POLYGON ((10.7248905532604...,"{'type': 'Polygon', 'coordinates': [[[10.72489..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,application/octet-stream,2889cc19-e388-4c87-8de5-9c14ee9909fa,S2A_MSIL1C_20180602T213531_N0500_R086_T05VPG_2...,application/octet-stream,759989767,2024-02-12T05:58:58.148000Z,2024-02-12T07:12:15.587361Z,2025-06-21T04:58:22.002739Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/02/S2...,"[{'Value': '11d44efc9f5583c54c2b343093ea5528',...","{'Start': '2018-06-02T21:35:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-149.22517 60.09...,"{'type': 'Polygon', 'coordinates': [[[-149.225..."
1832,application/octet-stream,298324de-2721-446a-bb65-207de5a4d7e6,S2A_MSIL1C_20180601T151911_N0500_R068_T24XVF_2...,application/octet-stream,485791522,2024-02-12T04:22:50.724000Z,2024-02-12T06:44:54.752537Z,2025-06-21T05:18:57.621262Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C_N0500/2018/06/01/S2...,"[{'Value': 'f7b37340c1c46d924e99803270726a6e',...","{'Start': '2018-06-01T15:19:11.024000Z', 'End'...",geography'SRID=4326;POLYGON ((-38.70392 72.814...,"{'type': 'Polygon', 'coordinates': [[[-38.7039..."
1833,application/octet-stream,39e22993-23f1-42c7-bf38-40c42cc0c199,S2A_MSIL1C_20180602T213531_N0500_R0

From above, we can see that there are 342 rows for S3 OLCI and 1836 rows for S2. And next we will use these metadata to co-locate them and produce another table shows the details of the colocation pairs.

## Co-locate the data

In this section we use the metadata we have just produced to produce the co-location pair details. The logic of the code is match rows from S2 and S3 OLCI by their geo_footprint.

In [ ]:
s3_olci_metadata = pd.read_csv(
    path_to_save_data + "sentinel3_olci_metadata.csv"
)
s2_metadata = pd.read_csv(
    path_to_save_data + "sentinel2_metadata.csv"
)

In [ ]:
s3_olci_metadata["ContentDate.Start"] = pd.to_datetime(
    s3_olci_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s3_olci_metadata["ContentDate.End"] = pd.to_datetime(
    s3_olci_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

s2_metadata["ContentDate.Start"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s2_metadata["ContentDate.End"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

results = check_collocation(
    s2_metadata, s3_olci_metadata, start_date, end_date,time_window=pd.to_timedelta("10 minutes")
)


As usual, you can have a look at the co-location output

In [ ]:
from IPython.display import display

display(results.head(5))


,Satellite1_Name,Satellite1_ID,Satellite1_Footprint,Satellite2_Name,Satellite2_ID,Satellite2_Footprint,Overlap_Start,Overlap_End
0,S2A_MSIL2A_20180601T102021_N0500_R065_T33VVG_2...,159afe10-6623-48f1-9e9c-8f7e01e54348,"{'type': 'Polygon', 'coordinates': [[[13.42031...",S3A_OL_1_EFR____20180601T101730_20180601T10203...,295324a8-8e11-3902-a77d-27d5b10878e7,"{'type': 'Polygon', 'coordinates': [[[-8.30793...",2018-06-01T10:17:30.170794,2018-06-01T10:20:30.170794
1,S2A_MSIL2A_20180601T102021_N0500_R065_T33VVG_2...,159afe10-6623-48f1-9e9c-8f7e01e54348,"{'type': 'Polygon', 'coordinates': [[[13.42031...",S3A_OL_1_EFR____20180601T101430_20180601T10173...,0ad625d8-6eee-3eb1-80ea-5d02f621ce95,"{'type': 'Polygon', 'coordinates': [[[-6.79013...",2018-06-01T10:14:30.170794,2018-06-01T10:17:30.170794
2,S2A_MSIL2A_20180601T102021_N0500_R065_T33VVG_2...,159afe10-6623-48f1-9e9c-8f7e01e54348,"{'type': 'Polygon', 'coordinates': [[[13.42031...",S3B_OL_1_EFR____20180601T101626_20180601T10192...,7f2bc46d-ab62-3ba2-a478-392d561c6f25,"{'type': 'Polygon', 'coordinates': [[[-8.15625...",2018-06-01T10:16:25.986338,2018-06-01T10:19:25.986338
3,S2A_MSIL2A_20180601T102021_N0500_R065_T33VVG_2...,159afe10-6623-48f1-9e9c-8f7e01e54348,"{'type': 'Polygon', 'coordinates': [[[13.42031...",S3B_OL_1_EFR____20180601T101326_20180601T10162...,a7e3dc02-3ffa-36f2-a498-d521ab6addca,"{'type': 'Polygon', 'coordinates': [[[-6.63156...",2018-06-01T10:13:25.986338,2018-06-01T10:16:25.986338
4,S2A_MSIL1C_20180601T001611_N0500_R059_T59WPT_2...,ea293a96-1e9d-489c-a59f-b4b330d78f97,"{'type': 'Polygon', 'coordinates': [[[173.6542...",S3A_OL_1_EFR____20180601T000835_20180601T00113...,89471a11-7232-3eda-9c31-1a954d03cda0,"{'type': 'MultiPolygon', 'coordinates': [[[[18...",2018-06-01T00:08:34.751541,2018-06-01T00:11:34.751541


With code below, you can visualise the co-located footprint.

In [ ]:
from IPython.display import display

map_result = plot_results(results.head(1))
display(map_result)

### Proceeding with Sentinel-3 OLCI Download

Moving forward, we turn our attention to downloading the Sentinel-3 OLCI data. The process mirrors the approach we took with Sentinel-2, maintaining consistency in our methodology. We'll apply the same logic of filename conversion and follow the structured steps to retrieve the data from the Copernicus dataspace.

In [ ]:
download_dir = ""  # Replace with your desired download directory
product_id = results['Satellite1_ID'][0] # Replace with your desired file id
file_name = results['Satellite1_Name'][0]# Replace with your desired filename
# Download the single product
# download_single_product(product_id, file_name, access_token, download_dir)

## Sentinel-3 SRAL

It is also possible to co-locate S2/S3 OLCI with S3 SRAL (altimetry data). The overall logic is the same, we just need to fetch the S3 SRAL metadata.

In [ ]:
sentinel3_sral_data = query_sentinel3_sral_arctic_data(
    start_date, end_date, access_token
)

sentinel3_sral_data.to_csv(
    path_to_save_data + "s3_sral_metadata.csv",
    index=False,
)

And now you do the co-locaton again for S3 SRAL with S2 for example.

In [ ]:
s3_sral_metadata = pd.read_csv(
    path_to_save_data + "s3_sral_metadata.csv"
)
s2_metadata = pd.read_csv(
    path_to_save_data + "sentinel2_metadata.csv"
)

In [ ]:
s3_sral_metadata["ContentDate.Start"] = pd.to_datetime(
    s3_sral_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s3_sral_metadata["ContentDate.End"] = pd.to_datetime(
    s3_sral_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

s2_metadata["ContentDate.Start"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["Start"])
).apply(make_timezone_naive)
s2_metadata["ContentDate.End"] = pd.to_datetime(
    s2_metadata["ContentDate"].apply(lambda x: eval(x)["End"])
).apply(make_timezone_naive)

results = check_collocation(
    s2_metadata, s3_sral_metadata, start_date, end_date,time_window=pd.to_timedelta("10 minutes")
)


And now you can plot the co-location results again.

In [ ]:
from IPython.display import display

map_result = plot_results(results.head(5))
display(map_result)